# Plate Mapper

<!-- ![Sexton Labs](images/sextonlabs.png =336x410)
 -->
<img src="images/sextonlabs.png" width="20%">

## Instructions

1. run first cell and selected inputs `csv` and output directory
2. Fill out UI elements for compound
3. Hit the go button
4. Repeat for other compounds
5. Run other cells, I'll add a button when I care
6. Change save name for csv and run cell

In [ ]:
import platemapper
import pandas as pd
import re # REEEEE

from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *
from ipyfilechooser import FileChooser

import seaborn as sns
import matplotlib as plt


sfc = FileChooser()
sfc.title = 'Select source csv'
fc = platemapper.make_chooser('output directory')
display(sfc)
display(fc)

In [ ]:
def make_compound(destplate, max_back, dilute, sId, sW, comp, startCol, startRow,  steps=5):
    """
    Function for button to make a compound
    args:
        destplate - str destination plate ID
        max_back - float max backfill vol
        dilute - str dilution step 1/2, log2 etc. NOTE: only 1/2 is implemented
        sID - str source plate ID
        sW - str source well ID
        comp - str compound name
        (startCol, startRow) - tuple of first col and row of compound col:[A-P][0-15], row:[2-24]
        steps - int number of dilution steps
    retruns:
        dc - dataframe of compound
    """
    transVol, backVol = platemapper.cal_transVol(max_back, dilute, steps)
    destWell = platemapper.get_wellIDs(startCol, steps, startRow)
    dc = platemapper.generate_df(sourceId=sId, sourceWell=sW, destID=destplate, compound=comp, steps=steps, 
                                 destWell=destWell, transVol=transVol, backVol=backVol)
    return dc

def button_handler(sender):
    """
    Handler function for submit button
    pulls values from the 'ui' and calls make_compound 
    then loads the dataframe into the compounds list
    """
    # values from sourceplate
    df = sp.loc[sp['ProductName'] == nameDrop.value]
    sId, sW, comp, = df['384PlateName'].values[0],df['PlateLocation'].values[0],df['ProductName'].values[0]
    
    # user input values
    dilute = diluteDrop.value
    destplate = destField.value
    max_back = int(float(fillField.value))
    dilute = diluteDrop.value
    
    col = colDrop.value
    row = rowDrop.value
    
    # add that bad boi
    compounds.append(make_compound(destplate, max_back, dilute, sId, sW, comp, col, row))
    print(f'Added Compound: {comp}')

    
source = sfc.selected
out = fc.selected_path
sp = pd.read_csv(source, encoding='utf-8')
sp.head()

In [ ]:

# source Compound
nameDrop = Dropdown(description='Compound',options=sp['ProductName'], value=None)

# dilution step
diluteDrop = Dropdown(description='Select Dilute amount',options=['log2', '1/2'], value=None)

# destination plate id
destField = Text(value='Destination Plate', description='Destination Plate')

# max transfer vol
fillField = Text(value='0.0', description='Transfer Vol')


# row and col selectors

rowDrop = Dropdown(
    options = [('A',0),('B',1),('C',2),('D',3),('E',4),('F',5),('G',6),('H',7),('I',8),('J',9),('K',10),('L',11),('M',12),('N',13),('O',14),('P',15)],
    value = 0,
    description = 'Row'
)
colDrop = Dropdown(
    options = [(x+1,x) for x in range(2, 22)],
    value= 3,
    description = 'col'
)


# go button
submit = Button(
    description='Add Compound',
    disabled=False,
    button_style='',
    tooltip='Only press when ready',
    icon=''
)

# lsit to hold compounds dataframes
compounds = []

# plate df, FIXME
plate = None

# bind button to handler
submit.on_click(button_handler)

# show me the bits
display(nameDrop)
display(diluteDrop)
display(destField)
display(fillField)
display(colDrop)
display(rowDrop)

display(submit)

In [ ]:
well_rows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'] # non-control rows
well_cols = [str(x) for x in range(1, 25)] # non-contorl columns

def make_control(destID):
    """
    creates controls and stores them in a dataframe
    assumes that control schema is universal
    args:
        destID - The Plate ID
    returns:
        df - dataframe of positvie and negative controls 
    """
    neg_row, pos_row = [],[]
    for y in well_rows:
        for x in ['1','2']:
            neg_row.append(y+x)
        for j in ['23','24']:
            pos_row.append(y+j)
            
    neg = pd.DataFrame.from_dict({
        'sourceID':['NC' for i in range(len(neg_row))],
        'sourceWell':['NC' for i in range(len(neg_row))],
        'destID':[destID for i in range(len(neg_row))], # destination plate ID
        'destWell':neg_row,
        'transferVol':[0.0 for i in range(len(neg_row))],
        'backfillVol':[0.0 for i in range(len(neg_row))],
        'compound':['NC' for i in range(len(neg_row))]
    })
    
    pos = pd.DataFrame({
        'sourceID':['PC' for i in range(len(pos_row))],
        'sourceWell':['PC' for i in range(len(pos_row))],
        'destID':[destID for i in range(len(pos_row))], # destination plate ID
        'destWell':pos_row,
        'transferVol':[0.0 for i in range(len(pos_row))],
        'backfillVol':[0.0 for i in range(len(pos_row))],
        'compound':['PC' for i in range(len(pos_row))]
    })
    
    df = pd.concat([neg, pos])
    
    return df

In [ ]:
def build_handler(*args):
    compounds.append(make_control(destField.value))
    return pd.concat(compounds)

build = Button(
    description='Make Plate',
    disabled=False,
    button_style='',
    tooltip='Press to add compounds to plate',
    icon=''
)

plate = build.on_click(build_handler, plate)
display(build)

In [ ]:
def find_miss():
    all_well = []
    for r in well_rows:
        for c in well_cols:
            all_well.append(r+c)
    
    miss = list(set(all_well)-set(plate['destWell'].unique()))
    
    miss_dat = pd.DataFrame({
        'sourceID':['EMPTY' for i in range(len(miss))],
        'sourceWell':['EMPTY' for i in range(len(miss))],
        'destID':[destField.value for i in range(len(miss))], # destination plate ID
        'destWell':miss,
        'transferVol':[0.0 for i in range(len(miss))],
        'backfillVol':[0.0 for i in range(len(miss))],
        'compound':['EMPTY' for i in range(len(miss))]
    })
    
    return miss_dat


In [ ]:
compounds.append(make_control(destField.value))
plate = pd.concat(compounds)
plate.head()

In [ ]:
test = pd.concat(compounds)

In [ ]:
display(HTML(plate.to_html()))

In [ ]:
plate2 = pd.concat([plate, miss_dat])

In [ ]:
plate2.head

In [ ]:
plate2.to_csv('full.csv', index=False)